# uproot essentials
A quick walkthrough the essentials operations performed with **uproot** to read and manipulate ROOT files in Python. 

https://uproot.readthedocs.io/en/latest/basic.html

In [2]:
import uproot
import awkward as ak
import numpy as np

## Explore the TFile

In [3]:
cms_opendata_file = "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/40000/14B6A8AE-C9FE-D744-80A4-DDE5D008C1CD.root"
file = uproot.open(cms_opendata_file)

In [4]:
file.classnames()

{'tag;1': 'TObjString',
 'Events;1': 'TTree',
 'LuminosityBlocks;1': 'TTree',
 'Runs;1': 'TTree',
 'MetaData;1': 'TTree',
 'ParameterSets;1': 'TTree'}

In [5]:
file.keys()

['tag;1',
 'Events;1',
 'LuminosityBlocks;1',
 'Runs;1',
 'MetaData;1',
 'ParameterSets;1']

Let's get the TTree using the file as a dictionary

In [6]:
events = file["Events"]

we can easily inspect all the TTree content

In [ ]:
events.show()

or get the type of all the branches

In [ ]:
events.typenames()

The number of "rows" or events is accessible as

In [8]:
events.num_entries

1434319

It's also possible to open directly the TTree from the file

In [9]:
events = uproot.open(f"{cms_opendata_file}:Events")
events

<TTree 'Events' (1504 branches) at 0x7af031135a50>

## Arrays from TTree

There are many ways to read data from TTrees in *uproot*. 

- Read a single branch to a numpy arrays or Pandas serier
- Read the full TTree data as a awkward-array
- Read partially the TTree by specifing a list of branch
- Apply a cut over the TTree rows before reading the data

In [ ]:
events.arrays?

In [16]:
met = events.arrays(["MET_pt", "MET_phi"], library="np", entry_stop=5000)

In [17]:
met

{'MET_pt': array([31.452927 , 40.65485  , 32.816944 , ..., 23.492601 ,  2.4211023,
        44.237286 ], dtype=float32),
 'MET_phi': array([-0.28570557, -2.8354492 , -0.5496826 , ...,  0.22387695,
         1.7602539 , -1.3271484 ], dtype=float32)}

If we request "flat" arrays, with library `np==numpy`, we get a dictionary of numpy arrays

In [18]:
E = events.arrays(["Electron_pt", "Electron_eta", "Electron_phi"], library="ak", entry_stop=5000)

By default the uproot.arrays() method will read the full data from the TFile. We will see later methods to iterate over chunks of data. 
For this demonstration let's read the first 5k events.

In [26]:
E

<Array [{Electron_pt: [], ... ] type='5000 * {"Electron_pt": var * float32, "Ele...'>

In [20]:
E.fields

['Electron_pt', 'Electron_eta', 'Electron_phi']

In [27]:
E.Electron_pt

<Array [[], [], [], ... 33.3, 32.3, 10.6], []] type='5000 * var * float32'>

This is a awkward array containing the requested branches.

## Iterating over entries

In [30]:
for batch in events.iterate(step_size=1000, entry_stop=10_000):
    print(repr(batch))

<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>
<Array [{run: 1, ... L1simulation_step: True}] type='1000 * {"run": uint32, "lum...'>


## Iterating over files

In [31]:
import yaml
with open("datasets.yaml") as f:
    datasets = yaml.safe_load(f)

In [ ]:
uproot.iterate?

In [36]:
for batch in uproot.iterate([ f"{file}:Events" for file in datasets["DYJetsToLL"]["files"][0:3]], 
                           filter_name=["Jet_pt", "Jet_eta", "Jet_phi"],
                           step_size=100_000):
    print(batch)

[{Jet_eta: [], Jet_phi: [], Jet_pt: []}, ... 0.945], Jet_pt: [48, 42.9, 25.3]}]
[{Jet_eta: [-2.25, -3.79, 2.86, 2.66, -3.13], ... Jet_pt: [39.9, 35.1, 22, 20.5]}]
[{Jet_eta: [1.23, 2.67], Jet_phi: [1.14, -2.85, ... -2.51], Jet_pt: [29.5]}]
[{Jet_eta: [0.446, -0.574, 2.14], Jet_phi: [-1.09, ... 57.6, 51.8, 21.4, 19.7, 16]}]
[{Jet_eta: [-2.76], Jet_phi: [1.22], Jet_pt: [16.1, ... Jet_phi: [], Jet_pt: []}]
[{Jet_eta: [-1.41, -1.45], Jet_phi: [-1.8, ... Jet_pt: [34.4, 29.4, 28.4, 15.5]}]
[{Jet_eta: [-0.616], Jet_phi: [1.94], Jet_pt: [, ... Jet_pt: [37.8, 32.2, 15.3]}]
[{Jet_eta: [-1.99, -1.19, -0.0124], Jet_phi: [, ... 55.5, 53.9, 27.9, 22.5, 15.8]}]
[{Jet_eta: [1.61, 1.42, 3.51, 4.42, -0.189, -3.76, ... 115, 93.9, 27.6, 24.1, 20.6]}]
[{Jet_eta: [0.0519, -1.16, -2.49, -3.79, -2.33], ... -0.62], Jet_pt: [17.4]}]
[{Jet_eta: [2.31, 1.78, -1.31, 2.38, 0.196], ... 59.6, 42.3, 36.4, 19.5, 17]}]
[{Jet_eta: [-1.33, -0.825, 2.23, 2.96], ... Jet_pt: [51.5, 21.1, 20.8, 16.9]}]
[{Jet_eta: [-2.81], Jet

## Writing objects to ROOT file

# Awkward arrays

Awkward arrays are very similar to **numpy** arrays, but they can accomodate multidimensional arrays with different size for each entry. 
For example: the pT of the Jets for a set of events can be represented as a 2D arrays, where each row (event) can have different number of columns (different number of jets). 

API reference: https://awkward-array.org/doc/1.10/api-reference.html

In [40]:
events = uproot.open(f"{cms_opendata_file}:Events")
df = events.arrays(entry_stop=5000)

In [55]:
jets_pt = df.Jet_pt

the `tolist()` method is useful to printout the full array for debugging and checking the structure

In [56]:
jets_pt[0:5].tolist()

[[],
 [48.1875, 47.84375, 16.28125],
 [45.0625, 27.578125, 16.078125],
 [21.0, 16.640625],
 [26.96875, 20.203125, 19.875, 19.703125]]

In [57]:
N_jets = ak.num(jets_pt)

In [46]:
N_jets

<Array [0, 3, 3, 2, 4, 2, ... 4, 2, 8, 3, 9, 4] type='5000 * int64'>

simple operations can be applied on the full arrays (as with numpy arrays). The **broadcast** rules works in the same way as numpy

In [53]:
jets_pt * 2

<Array [[], [96.4, 95.7, ... 37.6, 37.2, 36]] type='5000 * var * float32'>

In [54]:
jets_pt ** 2

<Array [[], [2.32e+03, ... 353, 345, 324]] type='5000 * var * float32'>

Also numpy functions can be applied

In [58]:
np.sqrt(jets_pt)

<Array [[], [6.94, 6.92, ... 4.34, 4.31, 4.24]] type='5000 * var * float32'>

## Indexing

In [60]:
jets_pt[1]

<Array [48.2, 47.8, 16.3] type='3 * float32'>

We can ask for a range of columns, and this works also if the events have a different lenght

In [63]:
jets_pt[0:10, :5]

<Array [[], [48.2, 47.8, ... 21.3], [43.2]] type='10 * var * float32'>

but if we request the first jet specifically, and it is not available for one of the event, an expection is raise. Have a look at the padding spection. 

In [64]:
jets_pt[0:10, 0]

ValueError: in ListOffsetArray64 attempting to get 0, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/cpu-kernels/awkward_NumpyArray_getitem_next_at.cpp#L21)

In [65]:
ak.firsts(jet_pt)

<Array [None, 48.2, 45.1, ... 39.4, 48.2, 51.4] type='5000 * ?float32'>

A useful trick is also getting an array representing the indexes of the elements of an array.

In [93]:
locidx = ak.local_index(jet_pt)
locidx

<Array [[], [0, 1, 2], ... 8], [0, 1, 2, 3]] type='5000 * var * int64'>

This is useful because indexing can also be done explicitely with an (awkward) arrays of indices. 

For example. 

In [170]:
jet_pt_1_5 = jet_pt[1:5]
jet_pt_1_5.tolist()

[[48.1875, 47.84375, 16.28125],
 [45.0625, 27.578125, 16.078125],
 [21.0, 16.640625],
 [26.96875, 20.203125, 19.875, 19.703125]]

We want to take the first jet of the first event, the second and third of the second events, and so on..

In [ ]:
jet_pt_1_5[ ak.Array([[0], [1,2], [1], [0,1]]) ]

## Splitting / combining columns

In [71]:
jet = ak.zip({"pt": df.Jet_pt, "eta": df.Jet_eta, "phi": df.Jet_phi})

In [72]:
jet.fields

['pt', 'eta', 'phi']

In [73]:
ak.unzip(jet)

(<Array [[], [48.2, 47.8, ... 18.8, 18.6, 18]] type='5000 * var * float32'>,
 <Array [[], [-1.48, ... 0.106, 3.7, 2.67]] type='5000 * var * float32'>,
 <Array [[], [1.27, ... -3.1, 0.846, -3.05]] type='5000 * var * float32'>)

## Masking
Masking == applying cuts. Masking can be applied at events level (on the axis 0, on rows), or at columns level. The masking syntax is equivalent to the numpy one. 

### All events with at least 2 jets

In [66]:
mask = ak.num(df.Jet_pt) >= 2

In [67]:
mask

<Array [False, True, True, ... True, True] type='5000 * bool'>

The mask is a 1D boolean array with the same size of the data arrays. 

In [68]:
events_2j = df[mask]

In [69]:
events_2j

<Array [{run: 1, ... L1simulation_step: True}] type='4410 * {"run": uint32, "lum...'>

The mask is applied to all the fields of the awkward arrays

### Keep only jets with at least 30 GeV

In [76]:
mask_jets = df.Jet_pt > 30

In [77]:
mask_jets

<Array [[], [True, ... False, False, False]] type='5000 * var * bool'>

In [78]:
mask_jets[0:10].tolist()

[[],
 [True, True, False],
 [True, False, False],
 [False, False],
 [False, False, False, False],
 [False, False],
 [True, False, False, False],
 [True, True],
 [True, False, False, False, False, False],
 [True]]

the mask is now 2D. 

In [83]:
jet_pt_masked = jet_pt[mask_jets]
jet_pt_masked

<Array [[], [48.2, 47.8], ... 30.4], [51.4]] type='5000 * var * float32'>

In [84]:
ak.min(jet_pt_masked)

30.015625

### Combining masks

In [89]:
mask_pt = df.Jet_pt > 30 
mask_eta = abs(df.Jet_eta) < 2
mask_tot = mask_pt & mask_eta

In [88]:
mask_tot

<Array [[], [True, ... False, False, False]] type='5000 * var * bool'>

In [91]:
masked_jet = df.Jet_pt[mask_tot]

OR 

mask_or = mask_pt | mask_eta

Negate

In [102]:
~mask_pt

<Array [[], [False, ... True, True, True]] type='5000 * var * bool'>

## Flattening / unflattening


Very often we need to "flatten" an array, to reduce it's dimensionality. For example we may need the full list of jets pt, without the event-by-event division, to apply some computation. 

We often need to reverse the flattening and reproduce the original dimensionality of the array.

In [110]:
flat_jet_pt = ak.flatten(jet_pt, axis=1)
flat_jet_pt

<Array [48.2, 47.8, 16.3, ... 18.8, 18.6, 18] type='16753 * float32'>

In [111]:
ak.unflatten(flat_jet_pt, N_jets)

<Array [[], [48.2, 47.8, ... 18.8, 18.6, 18]] type='5000 * var * float32'>

## Padding
Often we need to get a "regular" arrays, without any jagged structure. For example this is needed for machine learning applications. 
The missing information is usually **padded** with 0. or specific values. 

In [112]:
ak.to_regular(jet_pt)

ValueError: in ListOffsetArray64, cannot convert to RegularArray because subarray lengths are not regular

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)

we cannot convert it to a regular array because it is still jagged

In [120]:
padded_jet_pt = ak.pad_none(jet_pt, ak.max(N_jets))
padded_jet_pt[0:3].tolist()

[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [48.1875,
  47.84375,
  16.28125,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [45.0625,
  27.578125,
  16.078125,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None]]

In [126]:
ak.to_regular(padded_jet_pt)

<Array [[None, None, None, ... None, None]] type='5000 * 14 * ?float32'>

Note the change in the type field: the array has now a regular dimension of 14 elements, and it is no more "var" variable. 

None values are usually replaced with specific value like 0. 

In [128]:
padded_jet_pt_final = ak.fill_none(padded_jet_pt, 0.)
padded_jet_pt_final[0:5].tolist()

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [48.1875,
  47.84375,
  16.28125,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [45.0625,
  27.578125,
  16.078125,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [21.0, 16.640625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [26.96875,
  20.203125,
  19.875,
  19.703125,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]]

## Conversion to numpy

When converting an awkward array (regular) to numpy, if Nones are present, a masked numpy array is created

In [131]:
ak.to_numpy(padded_jet_pt)

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [48.1875, 47.84375, 16.28125, ..., --, --, --],
        [45.0625, 27.578125, 16.078125, ..., --, --, --],
        ...,
        [39.4375, 21.3125, 19.90625, ..., --, --, --],
        [48.15625, 40.84375, 38.03125, ..., --, --, --],
        [51.4375, 18.796875, 18.578125, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32)

The option `allow_missing` is used to check explicitely for None values and fail the conversion

In [133]:
ak.to_numpy(padded_jet_pt, allow_missing=False)

ValueError: ak.to_numpy cannot convert 'None' values to np.ma.MaskedArray unless the 'allow_missing' parameter is set to True

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/awkward/operations/convert.py#L306)

In [134]:
ak.to_numpy(padded_jet_pt_final, allow_missing=False)

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [48.1875  , 47.84375 , 16.28125 , ...,  0.      ,  0.      ,
         0.      ],
       [45.0625  , 27.578125, 16.078125, ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [39.4375  , 21.3125  , 19.90625 , ...,  0.      ,  0.      ,
         0.      ],
       [48.15625 , 40.84375 , 38.03125 , ...,  0.      ,  0.      ,
         0.      ],
       [51.4375  , 18.796875, 18.578125, ...,  0.      ,  0.      ,
         0.      ]])

## More complicated operations

### All possible combinations of 2 collections

In [142]:
mask_1j_1e = (ak.num(df.Jet_pt, axis=1)>0) & (ak.num(df.Electron_pt, axis=1)>0)
myevents = df[mask_1j_1e]

In [147]:
first_index , second_index = ak.unzip(ak.argcartesian([myevents.Jet_pt, myevents.Electron_pt]))

In [148]:
first_index[0:3].tolist()

[[0], [0, 0, 1, 1], [0, 1, 2]]

In [149]:
second_index[0:3].tolist()

[[0], [0, 1, 0, 1], [0, 0, 0]]

In [150]:
jets_comb = myevents.Jet_pt[first_index]
ele_comb = myevents.Electron_pt[second_index]

Now we have an array of jets and one of electrons, correctly aligned to be able to perform operations on them. 

In [154]:
diff_pt = abs(jets_comb - ele_comb)
diff_pt

<Array [[19], [3.7, ... 6.13, 17, 16, 5.66]] type='2051 * var * float32'>

In [155]:
diff_pt[0:3].tolist()

[[18.971145629882812],
 [3.7016525268554688,
  5.485103607177734,
  2.1079025268554688,
  3.8913536071777344],
 [10.823930740356445, 3.7614307403564453, 2.8864307403564453]]

### Taking the largest element

We can now look for the larget diff_pt element for each event. 
The arrays are already properly formatting to be able to perform the following

In [159]:
max_diff_pt_idx = ak.argmax(diff_pt, axis=1)
max_diff_pt_idx

<Array [0, 1, 0, 1, 8, 1, ... 0, 12, 14, 0, 2] type='2051 * ?int64'>

This is the index of the jet-electron combination with the largest difference in pt for each event.  We can use this indexing to get the correct jet for each event.

In [160]:
jets_comb[max_diff_pt_idx]

<Array [[31.8], [39.9, ... [38.2, 31.1, 30.2]] type='2051 * option[var * float32]'>

# Exercise time